<a href="https://colab.research.google.com/github/pedromedeiros1418/Resid-ncia/blob/main/modelo_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import arff
import matplotlib.pyplot as plt

# Classes do modelo
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier  
#from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.svm import SVC

# Funções de avaliação dos modelos
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, make_scorer, roc_auc_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

#Load the data using "arff.loadarff" then convert it to dataframe
data = arff.loadarff('dataset_30_page-blocks.arff')

In [ ]:
df = pd.DataFrame(data[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5473 entries, 0 to 5472
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   height    5473 non-null   float64
 1   lenght    5473 non-null   float64
 2   area      5473 non-null   float64
 3   eccen     5473 non-null   float64
 4   p_black   5473 non-null   float64
 5   p_and     5473 non-null   float64
 6   mean_tr   5473 non-null   float64
 7   blackpix  5473 non-null   float64
 8   blackand  5473 non-null   float64
 9   wb_trans  5473 non-null   float64
 10  class     5473 non-null   object 
dtypes: float64(10), object(1)
memory usage: 470.5+ KB


In [ ]:
#tratamento de dados da coluna class - object para int
df['class'] = df['class'].astype(int)

#entradas
X = df.iloc[:,:-1].values
#rotulos
y = df.iloc[:,10].values

In [ ]:
#O modelo SVM não trabalha com probabilidades. Para isto, foi necessário habilitar
# a probabilidade do modelo para que consigamos gerar
model = SVC(probability=True)

parametros = {'C': [0.1, 1, 10, 100, 1000]}

#C - responsável por controlar o quão tolerante a erros de classificação será o modelo treinado

In [ ]:
# Random Trails
# Ter o mesmo resultado independentemente de quantas vezes seja executado
NUM_TRIALS = 5

for i in range(NUM_TRIALS):

  #Variáveis para relatório de classificação
  cls_original = []
  cls_predita = []

  #utilizado para dividir reparticionar o dataset para o GridSearchCV
  inner_cv = KFold(n_splits=10, shuffle=True, random_state=i) 

  #utilizado para reparticionar o X_train para avaliação do cross-validate
  outer_cv = KFold(n_splits=10, shuffle=True, random_state=i)

  #customização do relatório utilizando acurácia
  def classification_report_with_accuracy_score(y_true, y_pred):
    cls_original.extend(y_true)
    cls_predita.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score

  #GridSearchCV para encontrar os melhores parâmetros para o modelo
  clf = GridSearchCV(model, param_grid=parametros, cv=inner_cv)
  clf.fit(X,y)

  print(clf.best_params_)

  #Cross-validation aninhado com gridSearch - utiliza o melhor parâmetro para avaliar 
  #Reparticionando em 10-fold
  scores = cross_val_score(clf, X, y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

#Valores médios no relatório de classificação para todas as folds em uma validação cruzada 10-fold
print(classification_report(cls_original, cls_predita, digits=4))

{'C': 1000}
{'C': 1000}
{'C': 1000}
{'C': 1000}
{'C': 1000}
              precision    recall  f1-score   support

           1     0.9471    0.9921    0.9691      4913
           2     0.8354    0.6170    0.7098       329
           3     0.6000    0.2143    0.3158        28
           4     0.9600    0.2727    0.4248        88
           5     0.8163    0.3478    0.4878       115

    accuracy                         0.9404      5473
   macro avg     0.8318    0.4888    0.5814      5473
weighted avg     0.9361    0.9404    0.9313      5473



In [ ]:
print(confusion_matrix(cls_original, cls_predita))

In [ ]:
roc_auc_score(y, clf.predict_proba(X), multi_class="ovr")